In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.forecasting.theta import ThetaModel

from bikes.evaluate.split import train_test_split, TEST_PERIOD

In [ ]:
def plot_timeseries(actual: pd.Series, predicted: pd.Series):
    fig, ax = plt.subplots()

    ax.plot(actual, label="Observed")
    ax.plot(predicted, label="Predicted")

    ax.set(ylabel="Count")
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    ax.legend()

    fig.tight_layout();

    return ax

## Theta Model

In [ ]:
cycle_counts = pd.read_csv("cycle_counts.csv", parse_dates=["date"])

In [ ]:
LOCATION = "Quay Street Eco Display Classic"

location_df = cycle_counts.loc[cycle_counts["location"] == LOCATION].copy()
location_df = location_df.set_index("date").sort_index()
train_df, test_df = train_test_split(location_df)
y_train, y_test = train_df["count"], test_df["count"]

In [ ]:
fig, ax = plt.subplots()
ax.plot(y_train.iloc[-300:], label="Observed", lw=2)
ax.set(ylabel="Count")
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
ax.legend()
fig.tight_layout();

In [ ]:
theta = ThetaModel(y_train)
theta = theta.fit()

test_start, test_end = TEST_PERIOD
test_dates = pd.date_range(test_start, test_end, freq="D", inclusive="left")
forecasts = theta.forecast(steps=len(test_dates))
forecasts.index = test_dates

In [ ]:
plot_timeseries(y_test, forecasts)

In [ ]:
# Save forecasts
forecast_df = pd.merge(
    left=test_df.rename(columns={"count": "ytrue"}),
    right=forecasts.to_frame(name="yhat_theta"),
    left_index=True,
    right_index=True,
    how="left"
)
forecast_df = forecast_df.reset_index()

In [ ]:
assert not forecast_df["date"].isna().any()

In [ ]:
forecast_df.to_csv(f"./forecasts/theta/{LOCATION.replace(' ', '_').lower()}.csv")